# matrix problem to be solved

The follow code loads a linear system for a laplacian.

In [ ]:
""" this is a multi-basis vector version of GMRES"""

import csv
import scipy.sparse
from numpy import diag, zeros, zeros_like, loadtxt, array, matrix, dot, identity, isclose, NAN, empty
from numpy.linalg import inv, norm, det, solve

print("Loading matrices...")
        
rows = []
cols = []
vals = []
        
# AA = scipy.sparse.dok_matrix((100000, 100000))
csvreader = csv.reader(open('AA.dat'), delimiter=' ', skipinitialspace=True)
for line in csvreader:
    row, column, val = line[:3]
    rows.append(int( row ))
    cols.append(int( column ))
    vals.append(float( val ))
    # AA[int(row), int(column)] = float(val)

rows = array(rows)-1
cols = array(cols)-1
vals = array(vals)
A = scipy.sparse.coo_matrix((vals,( rows, cols )))
A = A.tocsr()

b = loadtxt(open('bb.dat'))
nn = len(b)

b = matrix(b).transpose()

print("Matrices loaded.")
print("A has dimensions: ",A.shape)
print("b has dimensions: ",b.shape)
#    print(A)
#    print(b)

# Multiple vector Generalized Minimum Residual Method

Given an initial guess $x_0$, we want to solve the equation:
$$ A x = b $$
where
$$x = x_0 + \delta x$$
and $\delta x$ is "small". Substituting for x gives:
$$A \delta x = (b-A x_0) = \rho$$
where $\rho$ is the residual.

The initial guess $x_0$ is not critical, using zero is acceptable.

In [ ]:
x0 = zeros((len(b),1))

Similarly to using $x_0$ as an initial guess for $ x $, we also consider a preconditioner $M$ as an easy to compute linear operator that approximates $A^{-1}$.

Left preconditioning:
$$ M^{-1}Ax=M^{-1}b $$
Doing left preconditioning solves for x directly, altering the residuals with the preconditioner.

For this algorithm, we use right preconditioning:
$$ Ax=A M M^{-1}x=A M y=b $$
$$ y = M^{-1} x $$
$$ x = M y $$

We use the simplest option, the Jacobi (diagonal) preconditioner, which is simply the diagonal of A.

In [ ]:
M = scipy.sparse.identity(len(b))
inv_diag = 1./A.diagonal()
M = scipy.sparse.spdiags(inv_diag, [0,], nn, nn)

In [ ]:
Minv = scipy.sparse.spdiags(A.diagonal(), [0,], nn, nn)
should_be_ident = Minv*M
should_be_ident.nnz

In [ ]:
ident = scipy.sparse.identity(A.shape[0])
diff = should_be_ident-ident
print("Check that Minv is the inverse of M: ",diff.max())

The algorithm is as follows:
For a given guess for $x$, we have the residual $\rho = A x-b$.

We now define a sequence of Krylov vectors
$$ K_0 = \rho $$
$$ K_i = A M K_{i-1} $$
for $ 1 \leq i \leq n $.

We approximate $y$ as a linear combination of $k_{i}$

$$ y = \sum_{i=0}^{n-1} \alpha_i K_i$$

and find $\alpha_i$ that minimize $\lvert A x - b \rvert = \lvert A M y(\alpha_i)-b \rvert $.

Since y is a power series of A,

$$ A M y = A M ( \sum_{i=0}^{n-1} \alpha_i K_i ) = \sum_{i=0}^{n-1} \alpha_i A M K_i = \sum_{i=0}^{n-1} \alpha_{i} K_{i+1} $$

$$ 0=\frac{\partial}{\partial \alpha_i} ((A M y-b) \cdot (A M y-b)) = 2 (A M y-b) \cdot \frac{\partial (A M y)}{\partial \alpha_i} = 2 (A M y-b) \cdot K_{i+1}$$
$$ ( \sum_{j=0}^{n-1} \alpha_{j} K_{j+1} ) \cdot K_{i+1} = b \cdot K_{i+1}$$
$$ \sum_{j=0}^{n-1} \alpha_{j} (K_{i+1} \cdot K_{j+1}) = b \cdot K_{i+1}$$
defining 
$$ N_{ij} = K_i \cdot K_j $$
Then $ \alpha_i $ can be found by solving the n by n linear system:
$$ \sum_{j=0}^{n-1} N_{ij} \alpha_j = b \cdot K_{i+1} $$



In [ ]:
def krylov(A, b, x):
    "computes new iterated version of x from A and b"

    res = b - A*x

    # construct first krylov vectors
    k[0] = res
    print("norm(k[",0,"] = ",norm(k[0],ord=2))
    
    # construct the other n-1 krylov vectors
    for i in range(1, len(k)):
        k[i] = A*(M*k[i-1])
        print("norm(k[",i,"] = ",norm(k[i],ord=2))

    r = empty(len(k))
    r.fill(NAN)
    N = empty((len(k), len(k)))
    N.fill(NAN)
    for i in range(len(k)):
        r[i] = dot((A*k[i]).transpose(), res)
        for j in range(1, len(k)+1):
            N[i-1, j-1] = dot(k[i].transpose(), k[j-1])
            
    # print("r is ",r)
    # print("N is ",N)

    alpha = solve(N, r)

    # err = N*alpha - r
    # dalpha = solve(N, err)
    # alpha -= dalpha
    # err = N*alpha - r.transpose()

    y = zeros_like(x)
    for i in range(len(k)):
        y += alpha[i]*k[i]
    x += M*y


In [ ]:
N = 10
print("Initializing list of ",N,"Krylov vectors...")

k = []
for _ in range(N):
    kvec = array((len(b), 1))
    kvec.fill(NAN)
    k.append(kvec)

In [ ]:
x0 = zeros((len(b),1))
x = x0
print(x.max())
bNorm = norm(b, ord=2)
print("INITIALLY X IS ",norm(x,ord=2))
print("INITIALLY b IS ",norm(b,ord=2))

print("Initial Residual: ", norm(A*x - b, ord=2)/bNorm)
    
for _ in range(0, 5):
    print("ITERATION ", _)
    krylov(A, b, x)
    norm_resid = norm(A*x-b)/bNorm
    print("rho/originalresidual = ",norm(A*x-b, ord=2),"/",bNorm," = ",norm_resid)
    if norm_resid < 0.001:
        break